<a href="https://colab.research.google.com/github/VSusko/IA/blob/main/Apresentacao-disciplina/Agentes-implementacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementação - Data Center**

A seguir, será apresentada a implementação da funcionalidade de uma *smart house* para um Data Center. O objetivo deste código é, a partir de uma simulação, comparar o desempenho da aplicação da média móvel e da média simples para determinar a compra de suprimentos para o Data Center. 

O ambiente possui as seguintes características:
* A bateria do Data Center tem uma capacidade máxima de 500KWh
* Todos os dias uma certa quantidade aleatória de cargas é consumida no Data Center, e este valor flutua aleatoriamente entre 200 e 400KWh
* Todos os dias o preço do KWh varia entre `R$0,50` e `R$2,00`
* No primeiro dia, o Data Center possui uma carga inicial de 300KWh
* Se a bateria atingir 0 KWh, o Data Center desliga, ou seja, a simulação é interrompida

Assim sendo, o agente da smart house deve decidir diariamente comprar ou não mais cargas para suprir o Data Center dadas as condições:
* O agente comprará cargas se o preço atual do KWh é menor do que a média (simples ou móvel) dos últimos dias. No caso da média móvel, serão considerados apenas os últimos 5 dias, enquanto a média simples considera a média de todos os dias já decorridos
* O agente também comprará cargas se o estoque do Data Center atingir um valor menor ou igual a 200Kwh
* Quando o agente opta por comprar cargas, ele comprará até 400Kwh. A quantidade comprada dependerá do estoque atual do Data Center, sendo que se ele possuir valor maior que 100KWh, como a capacidade máxima da bateria é de 500Kwh, então será comprada apenas a diferença para completar o estoque da bateria
* Caso nenhuma das condições de compra seja satisfeita, o agente não comprará nada no dia



# Importação das bibliotecas necessárias para executar o código


In [ ]:
from random import *
import matplotlib.pyplot as plt
import numpy as np

# Definição das constantes do código

In [ ]:
# Definição das constantes
QTD_DIAS       = 20
NUM_SIMULACOES = 1000

# Funções de geração dos números aleatórios


In [ ]:
# Função que gera um vetor de numeros entre 200 e 400 para o consumo diario
def gerar_consumos():
  # Vetor que armazenará os valores para a simulação dos consumos diários
  consumos = []
  # São gerados 20 valores entre 200KWh e 400KWh
  for i in range(QTD_DIAS): 
    consumos.append(randint(200, 400))
  return consumos

# Função que gera um vetor de numeros entre 0.5 e 2 para o preço do Kwh
def gerar_precos():
  # Vetor que armazenará os valores para a simulação do preço do KWh
  precos = []
  # São gerados 20 valores entre R$0.50 e R$2.00
  for i in range(QTD_DIAS): 
    precos.append(0.5 + 1.5 * random())
  return precos

# Definição do ambiente

Vamos armazenar no ambiente as seguintes informações:

*   Informação do dia
*   Preço do produto em cada um dos dias da série
*   Histórico de preços
*   Histórico de estoque
*   Histórico com a quantidade de produtos comprados

In [ ]:
# Definicao da classe ambiente
class Ambiente():

  def __init__(self):
    # Ambiente explorado pelo agente de compra de papel higienico
    self.num_dias=0                       # Valor do numero de dias
    self.estoque=300                      # Valor do estoque inicial
    self.historico_preco=[1.5]            # Lista do estoque de preços
    self.historico_estoque=[self.estoque] # Lista do historico de estoques
    self.historico_qtde_comprados=[0]     # Lista da quantidade de produtos comprados
    self.precos_aleatorios = gerar_precos()     # Vetor do preco de cada dia
    self.consumos_aleatorios = gerar_consumos() # Vetor do consumo de cada dia

  # Função que retorna o preço atual do KWh
  def percebe_preco_atual(self):
    return self.historico_preco[len(self.historico_preco)-1]
  
  # Função que retorna o estoque atual
  def percebe_estoque(self):
    return self.historico_estoque[len(self.historico_estoque)-1]    

  # Função que simula o ambiente
  def run(self, dic_acoes, iteracao):
    '''Realizar alteracoes no ambiente: 
       Definir, aleatoriamente, uma quantidade de produtos consumidos
       Atualizar o historico do preco atual e do estoque.
       Essas informacoes serao utilizadas pelo agente para decidir a compra ou nao de produtos
    '''
    # Consumo realizado (valores gerados aleatoriamente)
    print(f"Estoque atual: {self.historico_estoque[len(self.historico_estoque)-1]}")
    print(f"Consumo realizado no dia: {self.consumos_aleatorios[iteracao]}") # novo valor da quantidade consumida
    estoque_atual = self.historico_estoque[len(self.historico_estoque)-1] - self.consumos_aleatorios[iteracao] + dic_acoes["comprar"]
    print(f"Estoque final após consumo e compra: {estoque_atual}")
    
    self.historico_estoque.append(estoque_atual)               # Adicionando o estoque atual no histórico
    self.historico_qtde_comprados.append(dic_acoes["comprar"]) # Adicionando quantidade de carga comprada no histórico

    # Informando valor do produto no periodo (Atualizacao para o proximo dia)
    self.historico_preco.append(self.precos_aleatorios[iteracao]) # novo valor do produto, obtido pelo vetor de precos.
    print(f"Novo valor do preço: {self.precos_aleatorios[iteracao]}")

# Definição do agente

O agente vai armazenar informações sobre:
*   Número de dias já simulados
*   Ambiente (Composição de classes)
*   Informação sobre o estoque atual de cargas
*   Informação sobre o total de dinheiro gasto na simulação
*   Informação do preço atual do KWh
*   Valor da média (simples ou móvel) do preço do KWh

In [ ]:
# Definição da classe agente
class Agente():
  
  def __init__(self, ambiente):
    self.num_dias = 1                                               # Definição do número de dias
    self.ambiente= ambiente                                         # Copia do objeto ambiente
    self.estoque= ambiente.percebe_estoque()                        # Definição do estoque inicial
    self.total_gasto = 0                                            # Variável para armazenar a quantidade total gasta
    self.preco_atual = self.media = ambiente.percebe_preco_atual()  # Variável para o preço atual do KWh

  # Função que executa o agente, decidindo ou não comprar mais cargas 
  def executa_agente(self, media_movel):
    
    # Loop principal
    for i in range(QTD_DIAS): 
      # O agente percebe o estado do ambiente
      print(f"Dia: {i+1}")
      self.estoque= self.ambiente.percebe_estoque()         # Obtenção do novo estoque
      self.preco_atual= self.ambiente.percebe_preco_atual() # Obtenção do novo preço
      
      '''
        Controlador do agente:
        - Define a regra para compra de produtos:
          Se o preço atual for menor que a média ou o estoque estiver abaixo de 100, são comprados 300KWh.
          
          Se a soma do estoque atual com 300 for maior do que o limite superior do DataCenter (500KWh),
          então será comprado a quantidade para completar o estoque.
      '''
      if (self.preco_atual < self.media) or (self.estoque <= 200):
        if (self.estoque + 300) <= 500: 
          compra= 400
        else:
          compra= 500 - self.estoque
      else:
        compra= 0
      
      print(f"Compra = {compra}")
      
      # Fim do controlador
      self.total_gasto += self.preco_atual*compra
      # O agente aplica modificacoes ao ambiente)
      self.ambiente.run({"comprar": compra}, i)
      # Aumentando o contador de dias
      self.num_dias+=1
      # Cálculo das médias: o primeiro caso representa a média móvel, e o segundo a média simples
      if media_movel and self.num_dias > 5:
        self.media = (self.media*(5) + self.preco_atual - self.ambiente.historico_preco[(self.num_dias - 1) - 5])/5
      else:
        self.media = (self.media*(self.num_dias-1) + self.preco_atual)/self.num_dias
        
      print(f"Media atual = {self.media}")
      
      print(f"\n")
      # Se o estoque zerar, a simulação termina
      if self.estoque <= 0 and i >= 1:
        return self.num_dias
      
    return self.num_dias

# Definção da classe para impressão dos resultados


In [ ]:
# Classe para a impressão dos gráficos
class Imprime:
  @staticmethod
  def imprime_resultado(agente):
    historico_dias = np.linspace(0, agente.num_dias, agente.num_dias)

    # Primeira impressão: historico do preco
    fig = plt.figure(figsize=(16,8))
    spec = fig.add_gridspec(1,3)

    axis1 = fig.add_subplot(spec[0,0])
    axis2 = fig.add_subplot(spec[0,1])
    axis3 = fig.add_subplot(spec[0,2])

    axis1.plot(historico_dias, agente.ambiente.historico_preco, 'bo--', label='Historico (preço)')
    axis1.legend()

    # Segunda impressão: historico qtde itens comprados
    axis2.vlines(historico_dias, ymin=0, ymax=agente.ambiente.historico_qtde_comprados)
    axis2.plot(historico_dias, agente.ambiente.historico_qtde_comprados, "go")
    axis2.set_ylim(0, 300)

    # Terceira impressão: historico do estoque
    axis3.plot(historico_dias, agente.ambiente.historico_estoque, 'rD--', label='Historico (estoque)')
    axis3.legend()
    plt.show()

# Colocações teóricas

Uma vez que o objetivo é comparar as duas médias, a métrica de comparação será o número de dias pelos quais o Data Center pôde operar sem interrupções e o dinheiro total gasto para cada cenário. Dessa forma, o código será executado 1000 vezes e será observado em qual das médias a bateria manteve-se ativa por mais tempo e quanto foi gasto para tal

Vale mencionar a diferença entre média simples e média móvel. A média simples constitui a razão entre a soma de todos os preços do KWh e o número de dias decorridos durante a simulação. Por outro lado, a média móvel corresponde à média dos preços do KWh apenas dos últimos 5 dias. Dessa forma, para cada dia depois do quinto dia de simulação a média do preço é calculada com base nos preços dos 5 dias anteriores. Os casos em que a simulação dura até 5 dias não são considerados na análise, uma vez que a média móvel possui uma janela de 5 dias, e portanto, as duas médias terão valores equivalentes nesse período, não sendo passível compará-las. 

Uma vez explicitadas tais diferenças, é possível inferir que a média móvel tende a fornecer valores mais coerentes para a compra de mais suprimentos, porque esta média acompanha os preços mais recentes do produto. Por outro lado, na média simples, preços antigos que distoam muito dos mais recentes podem ter uma influência maior no valor da média, resultando na escolha de não compra do agente. 

# Execução do programa

Por um determinado número de dias (iterações), veremos como o agente reagirá conforme o cálculo da média. Para centralizar a análise do agente nas médias, o ambiente simulado possui as mesmas caractéristicas, por isso, são usados os dois vetores globais anteriormente apresentados para armazenar os valores do preço do KWh e do consumo diário. Dessa forma, os dois agentes estarão condicionados ao mesmo ambiente. 

In [ ]:
# Agente da média simples
ambiente_atuacao_simples = Ambiente()
smart_house_media_simples = Agente(ambiente_atuacao_simples)
total_dias_simulados_simples = smart_house_media_simples.executa_agente(False, 20)
print(f"Total de dias simulados: {total_dias_simulados_simples}\n")
print(f"Total gasto: {smart_house_media_simples.total_gasto}\n")
Imprime.imprime_resultado(smart_house_media_simples)

# Agente da média móvel
ambiente_atuacao_movel = Ambiente()
smart_house_media_movel = Agente(ambiente_atuacao_movel)
total_dias_simulados_movel = smart_house_media_movel.executa_agente(True, 20)
print(f"Total de dias simulados: {total_dias_simulados_movel}\n")
print(f"Total gasto: {smart_house_media_movel.total_gasto}")
Imprime.imprime_resultado(smart_house_media_movel)

Para a obtenção dos resultados, teríamos que analisar qual das duas médias fornece um desempenho melhor no total gasto ou em durabilidade do Data Center. Com o código mostrado no tópico acima, esta tarefa pode ser feita manualmente, observando elementos do gráfico e registrando quando uma média teve performance melhor que outra. Os gráficos mostram resultados como: 

![alt text](Figure_1.png)
![alt text](Figure_2.png)

Entretanto, como podemos perceber, essa simulação não compreende uma simulação válida visto que durou apenas 3 dias e não podemos comparar as médias. Assim sendo, o fato de que podemos ter simulações descartáveis e que queremos obter uma análise estatística mais precisa, ou seja, com um número grande de simulações para obter resultados mais coerentes, uma nova implementação da última parte do programa foi realizada, apenas para que seja possível visualizar quantas vezes uma média foi superior à outra. 

O novo código não mostrará graficamente os resultados, porém será possível realizar as simulações de forma muito mais ágil. Além disso, serão contabilizadas apenas as simulações em que o número de dias ultrapassou 5. Assim sendo, os testes serão divididos da seguinte forma:
*   2000 simulações com um periódo de 20 dias no máximo em cada
*   2000 simulações com um periódo de 100 dias no máximo em cada
*   2000 simulações com um periódo de 200 dias no máximo em cada

Desse modo, o programa contará quantas vezes qual média produziu um gasto menor e quantas vezes qual média consguiu manter o Data Center ativo por mais tempo, bem como os casos de empate. 

In [ ]:
metrica_media_simples_dias = 0        # Variável que contabiliza quantas vezes a média simples durou mais tempo
metrica_media_simples_total_gasto = 0 # Variável que contabiliza quantas vezes a média simples produziu gastos menores
metrica_media_movel_dias = 0          # Variável que contabiliza quantas vezes a média móvel durou mais tempo
metrica_media_movel_total_gasto = 0   # Variável que contabiliza quantas vezes a média móvel produziu gastos menores
empate_total_gasto = 0 # Variável que contabiliza empate no dinheiro total gasto nas duas médias
empate_dias = 0 # Variável que contabiliza empate no número de dias das duas médias
simulacoes = 0 # Número de simulacoes totais
while True:
  # Agente da média simples
  ambiente = Ambiente()
  smart_house_media_simples = Agente(ambiente)
  total_dias_simulados_simples = smart_house_media_simples.executa_agente(False, 20)

  # Se o numero de dias simulado for menor ou igual a 5, recomece
  if total_dias_simulados_simples <= 5: 
    continue
  
  # Reset do ambiente
  ambiente.num_dias=0                       
  ambiente.estoque=300                      
  ambiente.historico_preco=[1.5]            
  ambiente.historico_estoque=[ambiente.estoque] 
  ambiente.historico_qtde_comprados=[0]     
  
  # Agente da média móvel
  smart_house_media_movel = Agente(ambiente)
  total_dias_simulados_movel = smart_house_media_movel.executa_agente(True, 20)
  
  # Se o numero de dias for menor ou igual a 5, recomece
  if total_dias_simulados_movel <= 5:
    continue
  
  print(f"Total de dias simulados (media simples): {total_dias_simulados_simples}\n")
  print(f"Total gasto (media simples): {smart_house_media_simples.total_gasto}\n")
  print(f"Total de dias simulados (media movel): {total_dias_simulados_movel}\n")
  print(f"Total gasto (media movel): {smart_house_media_movel.total_gasto}")
  
  ''' Se a media simples durou por mais tempo, o contador aumenta em 1 
      Se a media movel durou por mais tempo, o contador aumenta em 1
      Em caso de empate, nenhumas das duas aumenta e ele é contabilizado
  '''
  if total_dias_simulados_simples > total_dias_simulados_movel:
     metrica_media_simples_dias += 1
  elif total_dias_simulados_simples < total_dias_simulados_movel:
     metrica_media_movel_dias += 1
  else:
    empate_dias += 1
  
  ''' Se a media simples produziu menos gasto, o contador aumenta em 1 
      Se a media movel produziu menos gasto, o contador aumenta em 1
      Em caso de empate, nenhumas das duas aumenta e ele é contabilizado
  '''
  if smart_house_media_simples.total_gasto < smart_house_media_movel.total_gasto:
     metrica_media_simples_total_gasto += 1
  elif smart_house_media_simples.total_gasto > smart_house_media_movel.total_gasto:
     metrica_media_movel_total_gasto += 1
  else:
    empate_total_gasto += 1
  
  # Aumento do numero de simulacoes
  simulacoes += 1
  
  if simulacoes == NUM_SIMULACOES:
    break

print(f"Métrica media simples dias: {metrica_media_simples_dias}\n")
print(f"Métrica media móvel dias: {metrica_media_movel_dias}\n")
print(f"Métrica empate em dias: {empate_dias}\n")
print(f"Métrica media simples total gasto: {metrica_media_simples_total_gasto}\n")
print(f"Métrica media móvel total gasto: {metrica_media_movel_total_gasto}\n")
print(f"Métrica empate em total gasto: {empate_total_gasto}\n")
    

# Resultados

Para o cenário de 2000 simulações com um periódo de 20 dias no máximo em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples durou mais tempo: 99
*   Número de vezes em que a média móvel durou mais tempo: 53
*   Número de empates do tempo de duração do Data Center: 1848
*   Número de vezes em que a média simples gerou menos custo: 73
*   Número de vezes em que a média móvel gerou menos custo: 108
*   Número de empates do custo: 1819

Logo as estatísticas deste cenário são:
*   A média simples faz com que o Data Center dure mais em 4,95% dos casos
*   A média móvel faz com que o Data Center dure mais em 2,65% dos casos
*   As duas médias são iguais quanto à duração do Data Center em 92,4% dos casos
*   A média simples faz com que o Data Center tenha menos despesas em 3,65% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 5,4% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 90,95% dos casos

Para o cenário de 2000 simulações com um periódo de 100 dias no máximo em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples durou mais tempo: 108
*   Número de vezes em que a média móvel durou mais tempo: 83
*   Número de empates do tempo de duração do Data Center: 1809
*   Número de vezes em que a média simples gerou menos custo: 99
*   Número de vezes em que a média móvel gerou menos custo: 121
*   Número de empates do custo: 1780

Logo as estatísticas deste cenário são:
*   A média simples faz com que o Data Center dure mais em 5,4% dos casos
*   A média móvel faz com que o Data Center dure mais em 4,15% dos casos
*   As duas médias são iguais quanto à duração do Data Center em 90,45% dos casos
*   A média simples faz com que o Data Center tenha menos despesas em 4,95% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 6,05% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 89% dos casos

Para o cenário de 2000 simulações com um periódo de 200 dias no máximo em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples durou mais tempo: 103
*   Número de vezes em que a média móvel durou mais tempo: 83
*   Número de empates do tempo de duração do Data Center: 1814
*   Número de vezes em que a média simples gerou menos custo: 97
*   Número de vezes em que a média móvel gerou menos custo: 118
*   Número de empates do custo: 1785

Logo as estatísticas deste cenário são:
*   A média simples faz com que o Data Center dure mais em 5,15% dos casos
*   A média móvel faz com que o Data Center dure mais em 4,15% dos casos
*   As duas médias são iguais quanto à duração do Data Center em 90,7% dos casos
*   A média simples faz com que o Data Center tenha menos despesas em 4,85% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 5,9% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 89,25% dos casos

# Conclusões

Percebemos que tanto no quesito durabilidade do Data Center quanto no quesito economia, na maioria das vezes, as duas médias têm desempenho equivalente. Isto significa que as duas médias, para as variáveis aleatórias geradas, possuem resultados muito próximos, e dessa forma, o agente tende a executar o mesmo comportamento em ambos os casos. Um possível motivo para este comportamento é que as variáveis aleatórias possuem intervalos de geração dos valores pequeno para percebermos diferenças acentuadas entre as médias (o valor do preço por exemplo que flutua entre 0,5 e 2).
Além disso, vale pontuar que para todos os cenários testados, e em raros casos, a média simples fez com que o Data Center mantivesse ativo por mais tempo, ao passo que a média móvel promoveu um gasto total menor.